In [1]:
import pysam
from buseq import iter_bam_feature
import sys

In [5]:
file_bam = '/public/home/liuzj/scripts/pipeline/analyzeNGSData/results/step2_deDuplicates/Total3_sortedDedupBam.bam'
file_intron_pos = '/public/home/liuzj/data/Araport11/represent.proteinIdPos.txt'
fileout = '/public/home/liuzj/scripts/pipeline/analyzeNGSData/results/step6_calIrRatio/Total3_irRatio.tsv'
strand_flag = 1
min_overlap = 4
write_read_info = 1

In [6]:
fileout_read_info = fileout + ".readinfo.txt" if write_read_info else ""

In [7]:
fileout_read_info

'/public/home/liuzj/scripts/pipeline/analyzeNGSData/results/step6_calIrRatio/Total3_irRatio.tsv.readinfo.txt'

In [8]:
with open(fileout, 'w') as OUT:
        OUT.write("intron_id\tchr_name\tintron_start\tintron_end\tintron_strand\tintron_coverage_ratio\ta\tb\tab\tc\to\tt\tiratio\tsratio\toratio\to1ratio\to2ratio\to1_type\to1_count\to2_type\to2_count\tother_o\n")

In [ ]:
def iter_intron_read_type(file_bam, file_intron_pos, min_overlap=6, strand_flag=0, fileout_read_info=""):
    
    filter_tag=["unmapped", "remove_duplicated"]
    feature_aligns = iter_bam_feature(file_bam, file_intron_pos, 
                     filter_bam_params={"filter_tag": filter_tag}, up=0, down=0, method=2)
    if fileout_read_info:
        READ_INFO_OUT = open(fileout_read_info, 'w')
        READ_INFO_OUT.write("intron_id\tchr_name\tintron_start\tintron_end\tintron_strand\tflag\tstr_blocks\tis_read1\tstrand\tread_name\n")
    
    for [intron_id, chr_name, intron_start, intron_end, intron_strand], aligns in feature_aligns:

        #边界位置（包含）
        #第一个1表示是intron的左端，2表示intron的右端。
        #第二个1表示左区间，2表示右区间。
        #判定overlap是否够用的：
        xover11 = intron_start - min_overlap
        xover12 = intron_start + min_overlap - 1
        xover21 = intron_end - min_overlap + 1
        xover22 = intron_end + min_overlap
        
        intron_reads = [] #intron内部read，没有任何junction跨该内含子
        this_intron_data = []
        for na, start, end, strand, read in aligns:
            
            #1.如果是链特异性的，过滤掉方向不一致的
            if strand_flag == 1:
                if read.is_read1:
                    if strand == intron_strand: continue
                else:
                    if strand != intron_strand: continue
            elif strand_flag == 2:
                if read.is_read1:
                    if strand != intron_strand: continue
                else:
                    if strand == intron_strand: continue
            origin_read_blocks = get_read_blocks(read)
            
            #2.将block分离成三个部分
            left_blocks, blocks, right_blocks = trim_read_blocks(origin_read_blocks, intron_start, intron_end)
                            
            #3.提取覆盖intron的junction信息
            #junction, junction_start, junction_end用于后面的计算
            left_junction = ""
            right_junction = ""
            inner_junction = ""
            if left_blocks:
                junction_start = left_blocks[-1][1] + 1
                if blocks and (blocks[0][0] >= intron_start): left_junction = str(junction_start) + "-" + str(blocks[0][0] - 1)
            if right_blocks:
                junction_end = right_blocks[0][0] - 1
                if blocks and (blocks[-1][1] <= intron_end): right_junction = str(blocks[-1][1] + 1) + "-" + str(junction_end)
            if left_blocks and right_blocks and (not blocks):
                inner_junction = str(junction_start) + "-" + str(junction_end)
            if len(blocks) >= 2:
                for i in range(len(blocks)-1):
                    if inner_junction: inner_junction += ":"
                    inner_junction = str(blocks[i][1] + 1) + "-" + str(blocks[i+1][0] - 1)
            junction = left_junction
            if inner_junction:
                if junction: junction += ":"
                junction += inner_junction
            if right_junction:
                if junction: junction += ":"
                junction += right_junction
            
            #4. 计算read剪切类型
            if blocks:
                block_start = blocks[0][0]
                block_end = blocks[-1][1]
                
            if not blocks:
                #没有blocks，肯定有left_blocks和right_blocks。
                #肯定是有junction跨这个intron。有可能恰好是这个intron。
                #有可能比这个intron长。
                if junction_start == intron_start:
                    if junction_end == intron_end:
                        flag = ["c"]   #正确的剪切
                    else:
                        flag = ["o", "", "1"] #3'外侧剪切
                else:
                    if junction_end == intron_end:
                        flag = ["o", "1"] #5’ 外侧剪切
                    else:
                        #5'和3’外侧剪切 （注意这种有可能是Exon skipping）
                        flag = ["o", "1", "1"]
            elif len(blocks) == 1:
                if block_start < intron_start:
                    #表明a也可能是o|0,o|0|1的片段
                    #--------...............--------- gene structure
                    #      -------------------------  ["ab"]
                    #      ------------.....--------- ["o", "0"]  5' 内侧剪切
                    #      ------------.........----  ["o", "0", "1"] 5内3外
                    #      --------                   ["a"]
                    if block_end > intron_end:
                        flag = ["ab"]  #block跨整个intron
                    else:
                        if right_blocks:
                            if junction_end == intron_end:
                                flag = ["o", "0"]  #5' 内侧剪切
                            else:
                                flag = ["o", "0", "1"]
                        else:
                            flag = ["a"]
                else:
                    #o||0u除了可能是o||0外，也可能是o|||e1, o||1|e1
                    #o|1|0u除了可能是o|1|0外，也可能是o|1||e1, o|1|1|e1 
                    #o|0u|除了可能是o|0外，也可能是o|||e1, o|1||e1
                    #o|0u|1除了可能是o|0外，也可能是o||1|e1, o|1|1|e1
                    #i可以是多种可能
                    #不过后两者的概率较小
                    #--------...............--------- gene structure
                    #  ------.....--------------      ["o","", "0"] 3内
                    #  ------.....-----.....------    ["o","","","e1"] ES
                    #  ------.....-----.......-----   ["o","", "1", "e1"]
                    #  ------.....-----               ["o","", "0u"]
                    #
                    #  ---........-------------       ["o","1", "0"] 5外3内
                    #  ---........-----.....------    ["o","1","", "e1"]
                    #  ---........-----.......-----   ["o","1", "1", "e1"]
                    #  ---........-----               ["o","1", "0u"]
                    #
                    #             ---------------     ["b"]
                    #             -----.....------    ["o","0u"]
                    #             -----........---    ["o","0u", "1"]
                    #             ------              ["i"]
                    if left_blocks:
                        if junction_start == intron_start:
                            if block_end > intron_end:
                                flag = ["o","", "0"]
                            else:
                                if right_blocks:
                                    if junction_end == intron_end:
                                        flag = ["o","","","e1"] #exon skipping
                                    else:
                                        flag = ["o","", "1", "e1"]
                                else:
                                    flag = ["o","", "0u"]#表示intron 3'没覆盖到                                    
                        else:
                            if block_end > intron_end:
                                flag = ["o","1", "0"]
                            else:
                                if right_blocks:
                                    if junction_end == intron_end:
                                        flag = ["o","1","", "e1"]
                                    else:
                                        flag = ["o","1", "1", "e1"]
                                else:
                                    flag = ["o","1", "0u"]#表示intron 3’没覆盖到
                    else:
                        if block_end > intron_end:
                            flag = ["b"]
                        else:
                            if right_blocks:
                                if junction_end == intron_end:
                                    flag = ["o","0u"]
                                else:
                                    flag = ["o","0u", "1"]
                            else:
                                flag = ["i"]
            elif len(blocks) >= 2: 
                #--------...............--------- gene structure
                #     ------.......--------
                #  ------...------......---
                #  ----.....------........----
                e_num = len(blocks)
                flag = ["o", "", ""]
                if block_start < intron_start:
                    e_num -= 1
                    flag[1] = "0"
                else:
                    if left_blocks:
                        if junction_start < intron_start:
                            flag[1] = "1"
                        else:
                            flag[1] = ""
                    else:
                        flag[1] = "u"
                if block_end > intron_end:
                    e_num -= 1
                    flag[2] = "0"
                else:
                    if right_blocks:
                        if junction_end > intron_end:
                            flag[2] = "1"
                        else:
                            flag[2] = ""
                    else:
                        flag[2] = "u"
                if e_num:
                    flag.append("e" + str(e_num))
            
            #5. 计算两端覆盖是否满足条件
            #如果两侧有无关的block，说明比对可信。不用考虑覆盖问题。
            flag_overlap = True
            if flag[0] == "a":
                flag_left_block_overlap = (len(left_blocks) > 0) or (blocks[0][0] <= xover11)
                flag_right_block_overlap = blocks[0][1] >= xover12
                flag_overlap = flag_left_block_overlap and flag_right_block_overlap
            elif flag[0] == "b":
                flag_left_block_overlap =  blocks[0][0] <= xover21
                flag_right_block_overlap = (len(right_blocks) > 0) or (blocks[0][1] >= xover22)
                flag_overlap = flag_left_block_overlap and flag_right_block_overlap
            elif flag[0] == "ab":
                flag_left_block_overlap = (len(left_blocks) > 0) or (blocks[0][0] <= xover11)
                flag_right_block_overlap = (len(right_blocks) > 0) or (blocks[0][1] >= xover22)
                if flag_left_block_overlap:
                    if not flag_right_block_overlap:
                        flag[0] = "a"
                    else:
                        flag[0] = "ab"
                else:
                    if not flag_right_block_overlap:
                        flag[0] = "ab"
                        flag_overlap = False
                    else:
                        flag[0] = "b"
            elif flag[0] == "i":
                flag_overlap = True
            else:
                flag_left_block_overlap = False
                flag_right_block_overlap = False
                if left_blocks:
                    flag_left_block_overlap = (len(left_blocks)>1) or (left_blocks[0][1] - left_blocks[0][0] + 1 >= min_overlap)
                else:
                    if block_start < intron_start:
                        if block_start <= xover11:
                            flag_left_block_overlap = True
                    else:
                        if (blocks[0][1] - blocks[0][0] + 1) >= min_overlap:
                            flag_left_block_overlap = True
                if right_blocks:
                    flag_right_block_overlap = (len(right_blocks)>1) or (right_blocks[0][1] - right_blocks[0][0] + 1 >= min_overlap)
                else:
                    if block_end > intron_end:
                        if block_end >= xover22:
                            flag_right_block_overlap = True
                    else:
                        if (blocks[-1][1] - blocks[-1][0] + 1) >= min_overlap:
                            flag_right_block_overlap = True
                flag_overlap = flag_left_block_overlap and flag_right_block_overlap
            
            #6. 如果intron_strand为负，调整flag信息
            if flag[0] == "o":
                #一定不要放到下面的if语句里，因为intron_strand为+也要这样操作。要不然会导致不一致
                flag = flag + [""]*(4-len(flag))
            if intron_strand == "-":
                if flag[0] == "a":
                    flag = ["b"]
                elif flag[0] == "b":
                    flag = ["a"]
                elif flag[0] == "ab":
                    pass
                elif flag[0] == "c":
                    pass
                elif flag[0] == "i":
                    pass
                else:
                    ls5, ls3 = flag[2], flag[1]
                    flag[1] = ls5
                    flag[2] = ls3
            
            #7. 将flag由列表转化为字符串，暂时不考虑u，因此去掉u             
            flag = "|".join(flag)
            flag = flag.replace("u", "")
            
            #8. 将flag上添加上junction信息
            if junction and (flag[0] != "c"):
                flag = flag + ":" + junction
            
            #9. 记录intron_reads。用于计算intron coverage
            if flag == "a" or flag == "b" or flag == "ab" or flag == "i":
                intron_reads.append([block_start, block_end])
            
            #10. 如果两侧overlap达不到要求，则加上p
            if not flag_overlap:
                flag = "p" + flag
            # str(start), str(end), strand, 
            str_blocks = ":".join([",".join([str(s) + "-" + str(e) for s, e in left_blocks]),
                                   ",".join([str(s) + "-" + str(e) for s, e in blocks]),
                                   ",".join([str(s) + "-" + str(e) for s, e in right_blocks])])
            if fileout_read_info:
                READ_INFO_OUT.write("\t".join([intron_id, chr_name, str(intron_start), str(intron_end), intron_strand, 
                                    flag, str_blocks, str(read.is_read1), strand, read.query_name]) + "\n") 
            this_intron_data.append([flag, read.query_name, str_blocks])
            
            
        yield([intron_id, chr_name, intron_start, intron_end, intron_strand], this_intron_data, intron_reads)
    READ_INFO_OUT.close()